In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# importing libaries
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# importing dataset
from keras.datasets import cifar10
(X_train, y_train) , (X_test, y_test) = cifar10.load_data()

In [ ]:
print(y_train.shape)
print(X_train.shape)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical



In [ ]:
y_train =to_categorical(y_train)

y_test =to_categorical(y_test)

In [ ]:
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))

	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))

	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))

	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))

	model.add(Dense(10, activation='softmax'))

model.summary()

In [ ]:
# compile model

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# convert from integers to floats
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# normalize to range 0-1
X_train = X_train / 255.0
X_test = X_test/ 255.0

In [ ]:
# data augumentation
datagen = ImageDataGenerator(rotation_range=20,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              horizontal_flip=True)

datagen.fit(X_train)
flow=datagen.flow(X_train, y_train, batch_size=64)
steps = int(len(X_train) / 64)

In [ ]:
# fits the model on batches with real-time data augmentation:

history=model.fit_generator(flow , steps_per_epoch=steps , epochs=200 , validation_data= (X_test,y_test), verbose=0)

In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('%.3f' % (acc * 100.0))
print('%.3f' % (loss * 100.0))

In [ ]:
# make prediction.
# on test set
pred = model.predict(X_test)
print(pred)
print(y_test)

In [ ]:
history.history.keys()

In [ ]:
# accuracy plot for image classification
plt.plot(history.history['accuracy'],color='blue',label='train')
plt.plot(history.history['val_accuracy'],color='green',label='test')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'],color='blue',label='train')
plt.plot(history.history['val_loss'],color='green',label='test')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
Y_true = np.argmax(y_test, axis=1)
Y_true

In [ ]:
Y_pred_class= np.argmax(pred, axis=1) 
Y_pred_class

In [ ]:
labels=pd.read_csv('/kaggle/input/cifar-10/trainLabels.csv')
labels['label'].unique()

In [ ]:
# confusion matrix for 10 labels

cm=confusion_matrix(Y_true,Y_pred_class)
print(cm)

In [ ]:
names = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
from sklearn.metrics import confusion_matrix,classification_report

plt.subplots(figsize=(12,12))
# fmt for integer values
# cm is values of confusion matrix
# names for x and y are mentioned
plt.title('Prediction Count')
sns.heatmap(cm,xticklabels=names,yticklabels=names,annot=True,fmt='d')

plt.show()

In [ ]:
# showing result
# fixing space for each image
fig, axes = plt.subplots(4, 6, figsize=(10,10))
# defining boundary for each image
axes = axes.ravel()

for i in np.arange(0, 24):
    axes[i].imshow(X_test[i])
    axes[i].set_title("True: %s \nPredict: %s" % (names[Y_true[i]], names[Y_pred_class[i]]))
    plt.subplots_adjust(wspace=2)

In [ ]:
incorrect_pred_idx=np.where(Y_true!=Y_pred_class)[0].tolist()
incorrect_pred_idx

In [ ]:
fig,axes=plt.subplots(4,4,figsize=(10,10))
axes=axes.ravel()

for i in np.arange(0,16):
        axes[i].imshow(X_test[incorrect_pred_idx[i]])
        axes[i].set_title('True: %s \n Pred: %s'%(names[Y_true[incorrect_pred_idx[i]]],names[Y_pred_class[incorrect_pred_idx[i]]]))
        plt.subplots_adjust(wspace=1)
        

In [ ]:
print(classification_report(Y_true, Y_pred_classes))